In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)

In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), PL Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned PL data)
df = pd.read_csv('CdSe PL Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)

    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
144/144 [==============================] - 1s 3ms/step - loss: 0.0387 - val_loss: 0.0182
Epoch 2/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0212 - val_loss: 0.0172
Epoch 3/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0187 - val_loss: 0.0151
Epoch 4/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0173 - val_loss: 0.0152
Epoch 5/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0169 - val_loss: 0.0149
Epoch 6/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0160 - val_loss: 0.0147
Epoch 7/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0158 - val_loss: 0.0145
Epoch 8/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0155 - val_loss: 0.0144
Epoch 9/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0150 - val_loss: 0.0145
Epoch 10/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0154 - val_lo

144/144 [==============================] - 0s 1ms/step - loss: 0.0127 - val_loss: 0.0128
Epoch 82/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0127 - val_loss: 0.0128
Epoch 83/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0127 - val_loss: 0.0126
Epoch 84/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0124 - val_loss: 0.0130
Epoch 85/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0126 - val_loss: 0.0127
Epoch 86/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0125 - val_loss: 0.0131
Epoch 87/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0126 - val_loss: 0.0135
Epoch 88/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0127 - val_loss: 0.0130
Epoch 89/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0126 - val_loss: 0.0134
Epoch 90/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0127 - val_loss: 

Epoch 161/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0124
Epoch 162/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0123 - val_loss: 0.0123
Epoch 163/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0121 - val_loss: 0.0124
Epoch 164/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0126
Epoch 165/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0122
Epoch 166/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0123 - val_loss: 0.0123
Epoch 167/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0124
Epoch 168/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0121 - val_loss: 0.0122
Epoch 169/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0126
Epoch 170/300
144/144 [==============================] - 0s 1ms/step - lo

144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0125
Epoch 241/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0127
Epoch 242/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0121 - val_loss: 0.0123
Epoch 243/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0120 - val_loss: 0.0123
Epoch 244/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0120 - val_loss: 0.0124
Epoch 245/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0121 - val_loss: 0.0122
Epoch 246/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0121 - val_loss: 0.0123
Epoch 247/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0120 - val_loss: 0.0130
Epoch 248/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0122 - val_loss: 0.0132
Epoch 249/300
144/144 [==============================] - 0s 2ms/step - loss: 0.0122 - v

144/144 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0021
Epoch 20/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 21/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 22/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 23/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 24/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0021
Epoch 25/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 26/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 27/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 28/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 

...

144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 251/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 252/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 253/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 254/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 255/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 256/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 257/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 258/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 259/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - v

144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 30/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 31/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 32/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 33/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 34/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 35/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 36/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 37/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0018
Epoch 38/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 

144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 110/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 111/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 112/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 113/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0018
Epoch 114/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 115/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 116/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0018
Epoch 117/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 118/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0017 - v

144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 189/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 190/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 191/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 192/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 193/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 194/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 195/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 196/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 197/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - v

144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 268/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 269/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 270/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 271/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 272/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 273/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 274/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 275/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 276/300
144/144 [==============================] - 0s 1ms/step - loss: 0.0015 - v

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0965 0.0031
R2 (eV):	 0.5879 0.0350
RAE (eV):	 0.6281 0.0355
RMSE (eV):	 0.1221 0.0037
MAE (nm):	 22.0882 0.8092
R2 (nm):	 0.5728 0.0401
RAE (nm):	 0.6338 0.0369
RMSE (nm):	 28.0241 1.0049
Absorbance FWHM (direct)
MAE (eV): 	 0.0110 0.0009
R2 (eV):	 0.4997 0.0625
RAE (eV):	 0.6528 0.0390
RMSE (eV):	 0.0154 0.0010
MAE (nm):	 3.9531 0.2374
R2 (nm):	 0.4114 0.0627
RAE (nm):	 0.7342 0.0415
RMSE (nm):	 5.2712 0.3143
Absorbance FWHM (cascade)
MAE (eV): 	 0.0100 0.0007
R2 (eV):	 0.5788 0.0866
RAE (eV):	 0.5961 0.0431
RMSE (eV):	 0.0141 0.0013
MAE (nm):	 3.7631 0.2021
R2 (nm):	 0.4689 0.0868
RAE (nm):	 0.6999 0.0510
RMSE (nm):	 4.9849 0.2483


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0964 0.0030
R2 (eV):	 0.5886 0.0351
RAE (eV):	 0.6274 0.0355
RMSE (eV):	 0.1220 0.0037
MAE (nm):	 22.0669 0.8009
R2 (nm):	 0.5735 0.0401
RAE (nm):	 0.6332 0.0369
RMSE (nm):	 27.9993 0.9914
Absorbance FWHM (direct)
MAE (eV): 	 0.0110 0.0009
R2 (eV):	 0.5031 0.0634
RAE (eV):	 0.6539 0.0396
RMSE (eV):	 0.0154 0.0010
MAE (nm):	 3.9522 0.2343
R2 (nm):	 0.4142 0.0642
RAE (nm):	 0.7341 0.0422
RMSE (nm):	 5.2575 0.3110
Absorbance FWHM (cascade)
MAE (eV): 	 0.0100 0.0007
R2 (eV):	 0.5847 0.0827
RAE (eV):	 0.5955 0.0425
RMSE (eV):	 0.0140 0.0012
MAE (nm):	 3.7612 0.1990
R2 (nm):	 0.4729 0.0848
RAE (nm):	 0.6995 0.0501
RMSE (nm):	 4.9674 0.2464


In [12]:
np.savetxt("NN+MMCN_PL_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_cascade.csv", matrix_cascade, delimiter=",")